# Eigenvalues of the dielectric matrix

We compute a few eigenvalues of the dielectric matrix ($q=0$, $ω=0$) iteratively.

In [1]:
using DFTK
using Plots
using KrylovKit
using Printf

# Calculation parameters
kgrid = [1, 1, 1]
Ecut = 5

# Silicon lattice
a = 10.26
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

# Compute the dielectric operator without symmetries
model  = model_LDA(lattice, atoms, positions, symmetries=false)
basis  = PlaneWaveBasis(model; Ecut, kgrid)
scfres = self_consistent_field(basis, tol=1e-8);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.232797979692                   -0.50    6.0         
  2   -7.249367036029       -1.78       -1.39    1.0   10.0ms
  3   -7.250972694135       -2.79       -1.82    2.0   11.6ms
  4   -7.250953560013   +   -4.72       -1.87    2.0   12.1ms
  5   -7.251315577644       -3.44       -2.48    1.0   10.1ms
  6   -7.251337763936       -4.65       -3.06    1.0   10.0ms
  7   -7.251338657416       -6.05       -3.59    1.0   10.0ms
  8   -7.251338783761       -6.90       -3.89    2.0   11.8ms
  9   -7.251338796349       -7.90       -4.32    1.0   10.4ms
 10   -7.251338798482       -8.67       -4.97    2.0   11.9ms
 11   -7.251338798634       -9.82       -5.05    3.0   13.8ms
 12   -7.251338798692      -10.23       -5.55    1.0   10.5ms
 13   -7.251338798698      -11.23       -5.60    3.0   14.0ms
 14   -7.251338798704      -11.23       -6.22    1.0   10.5ms
 15   -7.

Applying $ε^† ≔ (1- χ_0 K)$ …

In [2]:
function eps_fun(δρ)
    δV = apply_kernel(basis, δρ; ρ=scfres.ρ)
    χ0δV = apply_χ0(scfres, δV)
    δρ - χ0δV
end;

… eagerly diagonalizes the subspace matrix at each iteration

In [3]:
eigsolve(eps_fun, randn(size(scfres.ρ)), 5, :LM; eager=true, verbosity=3);

[ Info: Arnoldi iteration step 1: normres = 0.06198435834205509
[ Info: Arnoldi iteration step 2: normres = 0.5158645562562821
[ Info: Arnoldi iteration step 3: normres = 0.8459982776164054
[ Info: Arnoldi iteration step 4: normres = 0.19587794323692703
[ Info: Arnoldi iteration step 5: normres = 0.40686045264409537
[ Info: Arnoldi schursolve in iter 1, krylovdim = 5: 0 values converged, normres = (1.47e-02, 3.02e-02, 3.49e-01, 2.06e-01, 1.02e-02)
[ Info: Arnoldi iteration step 6: normres = 0.37411436293354805
[ Info: Arnoldi schursolve in iter 1, krylovdim = 6: 0 values converged, normres = (4.99e-03, 2.61e-01, 2.25e-01, 1.03e-01, 3.41e-02)
[ Info: Arnoldi iteration step 7: normres = 0.09537329598050488
[ Info: Arnoldi schursolve in iter 1, krylovdim = 7: 0 values converged, normres = (2.53e-04, 2.84e-02, 6.29e-03, 6.85e-02, 5.49e-02)
[ Info: Arnoldi iteration step 8: normres = 0.12263535557645752
[ Info: Arnoldi schursolve in iter 1, krylovdim = 8: 0 values converged, normres = (1.33